In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 定义文件全局变量
path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/initdata/"

# 加载文件
file_order = pd.read_csv(path + "t_order.csv")
file_click = pd.read_csv(path + "t_click.csv")
file_user = pd.read_csv(path + "t_user.csv")
file_loan = pd.read_csv(path + "t_loan.csv")
file_loan_sum = pd.read_csv(path + "t_loan_sum.csv")

# 格式化时间
file_order['buy_time_format'] = pd.to_datetime(file_order['buy_time'])

# 创建索引
file_order = file_order.set_index('buy_time_format')

#聚合用户订单月份数据
def user_order_month_aggregate(month):
    # 取月份数据
    file_order_month = file_order[month]
    # 重置索引
    file_order_month = file_order_month.reset_index()
    # 统计用户订单某个类目的价格、数量、折扣
    file_order_month_sum = file_order_month.groupby(['uid','cate_id'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_sum = file_order_month_sum.reset_index()
    # 合并2列
    file_order_month_sum['cate_id_and_count'] = file_order_month_sum['cate_id'].astype('str') + ':' +  file_order_month_sum['qty'].astype('str')
    # 统计用户下单类目
    file_order_month_sum_unique_cate_list = file_order_month_sum.groupby('uid')['cate_id_and_count'].unique()
    # 重置索引
    file_order_month_sum_unique_cate_list = file_order_month_sum_unique_cate_list.reset_index()
    # 统计用户订单全部价格、数量、折扣
    file_order_month_total = file_order_month.groupby(['uid'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_total = file_order_month_total.reset_index()
    # 将数据进行聚合操作
    user_order_month_aggregate = pd.merge(file_order_month_sum_unique_cate_list,file_order_month_total,how='outer',on='uid')
    return user_order_month_aggregate

# 索引各个月份数据
file_order_month_aggregate_11 = user_order_month_aggregate('2016-11')
file_order_month_aggregate_10 = user_order_month_aggregate('2016-10')
file_order_month_aggregate_09 = user_order_month_aggregate('2016-09')
file_order_month_aggregate_08 = user_order_month_aggregate('2016-08')

# 合并
file_click['pid_param'] = file_click['pid'].astype('str') +'_' + file_click['param'].astype('str')

# 用户点击统计
user_click_count = pd.DataFrame(file_click.groupby(['uid','pid_param'])['uid'].count())

# 重命名统计列
user_click_count.columns = ['click_count']

# 查询某个用户的点击统计
def user_click_count_serch(uid):
   user_click_count_serch = user_click_count.xs(uid, level='uid')
   return user_click_count_serch

# 查询某个参数的用户点击
def pid_and_param_click_count_serch(pid_param):
   pid_and_param_click_count_serch = user_click_count.xs(pid_param, level='pid_param')
   return pid_and_param_click_count_serch

# 查询用户的点击统计
# user_click_count_serch(1)

# 查询某个参数的用户点击
# pid_and_param_click_count_serch('8_1')

# 重置索引,并设置索引为uid
user_click_count_detail = user_click_count.reset_index().set_index('uid')

# 统计用户的总点击次数
user_click_count_total = pd.DataFrame(user_click_count.groupby('uid')['click_count'].sum())

# 重命名列名称
user_click_count_total.columns = ['click_count_total']

# 重置索引
user_click_count_total = user_click_count_total.reset_index()

# 合并2列为字典
user_click_count_detail['param_and_click'] = user_click_count_detail['pid_param'].astype('str') + ':' + user_click_count_detail['click_count'].astype('str')

# 合并分组
user_click_count_detail_aggregate = pd.DataFrame(user_click_count_detail.groupby('uid')['param_and_click'].unique())

# 重置索引
user_click_count_detail_aggregate = user_click_count_detail_aggregate.reset_index()

# 用户点击聚合
user_click_count_aggregate = pd.merge(user_click_count_detail_aggregate,user_click_count_total,how='outer',on='uid')

# 各月用户点击和订单数据汇总
user_click_count_and_order_month_aggregate_08 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_08,how='outer',on='uid')
user_click_count_and_order_month_aggregate_09 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_09,how='outer',on='uid')
user_click_count_and_order_month_aggregate_10 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_10,how='outer',on='uid')
user_click_count_and_order_month_aggregate_11 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_11,how='outer',on='uid')

# 重置索引
user_click_count_detail = user_click_count_detail.reset_index()

# 统计用户最大点击
user_click_count_detail_max = user_click_count_detail.loc[user_click_count_detail.groupby(['uid'])['click_count'].idxmax()].sort_values(by='uid')

# 查询每月订单最大数量
def file_order_month_max(month):
    file_order_month_sum = file_order[month].reset_index().groupby(['uid','cate_id'])['price','qty','discount'].sum().reset_index()
    file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max = file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max.columns = ['uid','cate_id_by_max_qty','price_by_max_qty','qty_max','discount_by_max_qty']
    return file_order_month_sum_max

# 查询点击最多次数和购买数量最大
def click_count_max_and_order_qty_max(month):
    click_count_max_and_order_qty_max = pd.merge(user_click_count_detail_max,file_order_month_max(month),how='outer',on='uid')
    return click_count_max_and_order_qty_max

In [32]:
def loan_count(month):
    file_loan['loan_time_format'] = pd.to_datetime(file_loan['loan_time'])
    file_loan_index = file_loan.set_index('loan_time_format')
    file_loan_count = pd.DataFrame(file_loan_index[month].groupby('uid')['uid'].count())
    file_loan_count.columns = ['loan_count']
    file_loan_count = file_loan_count.reset_index()
    file_loan_sum = pd.DataFrame(file_loan_index[month].groupby(['uid'])['loan_amount'].sum())
    file_loan_sum.columns = ['loan_sum']
    file_loan_sum = file_loan_sum.reset_index()
    file_loan_count_and_sum = pd.merge(file_loan_count,file_loan_sum,how='outer',on='uid')
    file_loan_count_and_sum['loan_sum_mean'] = file_loan_count_and_sum['loan_sum']/file_loan_count_and_sum['loan_count']
    return file_loan_count_and_sum

In [28]:
#构建每个月的用户特征
def feature_month_uid(month):
    click_count_max_and_order_qty_max_tmp = click_count_max_and_order_qty_max(month)
    loan_count_tmp = loan_count(month)
    feature_month_uid = pd.merge(click_count_max_and_order_qty_max_tmp,loan_count_tmp,how='outer',on='uid')
    return feature_month_uid

In [34]:
pd.merge(feature_month_uid('2016-08'),file_user,how='outer',on='uid')

,uid,pid_param,click_count,param_and_click,cate_id_by_max_qty,price_by_max_qty,qty_max,discount_by_max_qty,loan_count,loan_sum,loan_sum_mean,age,sex,active_date,limit
0,1,8_1,66.0,8_1:66,22.0,73.158573,22.0,18.936605,NaN,NaN,NaN,35,1,2016-04-27,5.974677
1,4,10_16,50.0,10_16:50,22.0,14.665733,4.0,11.159862,2.0,10.445692,5.222846,30,1,2016-03-26,5.292154
2,5,8_1,7.0,8_1:7,1.0,52.248393,36.0,24.480130,NaN,NaN,NaN,30,2,2015-12-08,6.292055
3,6,10_28,112.0,10_28:112,36.0,11.949272,4.0,2.554230,NaN,NaN,NaN,25,1,2016-04-13,6.292055
4,7,10_16,33.0,10_16:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,1,2016-02-06,6.153414
5,8,10_16,81.0,10_16:81,1.0,43.810418,42.0,8.127342,NaN,NaN,NaN,35,1,2016-04-15,6.153414
6,9,10_28,5.0,10_28:5,26.0,11.585922,3.0,0.000000,NaN,NaN,NaN,40,1,2015-12-27,6.292055
7,11,8_1,7.0,8_1:7,9.0,9.054182,4.0,2.554230,NaN,NaN,NaN,25,1,2016-06-28,6.292055
8,14,10_16,67.0,10_16:67,26.0,2.866920,1.0,0.000000,NaN,NaN,NaN,30,1,2016-01-28,5.974677
9,16,6_5,14.0,6_5:14,14.0,10.476114,4.0,4.383337,NaN,NaN,NaN,30,1,2016-06-15,5.974677


In [35]:
file_loan = file_loan.set_index('loan_time_format')

In [38]:
file_loan = file_loan['2016-08'].reset_index()

In [39]:
file_loan['uid'].group

,loan_time_format,uid,loan_time,loan_amount,plannum
0,2016-08-03 00:05:26,12135,2016-08-03 00:05:26,3.862595,1
1,2016-08-03 00:13:25,41403,2016-08-03 00:13:25,5.584137,3
2,2016-08-03 00:13:58,74458,2016-08-03 00:13:58,4.723017,1
3,2016-08-03 00:19:33,12959,2016-08-03 00:19:33,3.862595,1
4,2016-08-03 00:23:13,89641,2016-08-03 00:23:13,4.292651,1
5,2016-08-03 00:24:32,3272,2016-08-03 00:24:32,3.862595,1
6,2016-08-03 00:36:19,90639,2016-08-03 00:36:19,4.544373,1
7,2016-08-03 00:50:20,66044,2016-08-03 00:50:20,4.622093,1
8,2016-08-03 01:55:12,89641,2016-08-03 01:55:12,4.292651,1
9,2016-08-03 02:26:52,41227,2016-08-03 02:26:52,5.070593,3
